# Hand digit classifier
---
## Incremental network quantization

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from skimage import io

In [8]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])

train_data = torchvision.datasets.MNIST(root='../', train=True, download=True, transform=transform)
test_set = torchvision.datasets.MNIST(root="../", train=False, download=True, transform=transform)

train_set = [train_data[i] for i in range(50000)]
validation_set = [train_data[i] for i in range(50000, 60000)]

In [9]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=2)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=64, shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=False, num_workers=2)

### Definig a VGG-7 inspired architecture model

In [10]:
class VGG7(nn.Module):
    def __init__(self):
        super(VGG7, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, padding="same", stride=1, bias=False)
        self.conv2 = nn.Conv2d(64, 64, 3, padding="same", stride=1, bias=False)
        self.conv3 = nn.Conv2d(64, 128, 3, padding="same", stride=1, bias=False)
        self.conv4 = nn.Conv2d(128, 128, 3, padding="same", stride=1, bias=False)
        
        self.fc1 = nn.Linear(7*7*128, 512, bias=False)
        self.fc2 = nn.Linear(512, 256, bias=False)
        self.fc3 = nn.Linear(256, 10, bias=False)

    def forward(self, x): 
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, stride=2)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, stride=2)  

        x = x.view(-1, 7*7*128)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)  

        x = F.log_softmax(x, dim=1)  

        return x

### Device initialization for training

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Training

In [13]:
net = VGG7()
net.to(device)

optimizer = optim.SGD(net.parameters(), lr = 1e-3, weight_decay=0)

logs_interval = 100
epochs = 5

train_losses = []
iteration = 0

net.train()

for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = net(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        iteration = iteration + 1
        if iteration % logs_interval:
            print('Train epoch:{}, batch index:{}, loss:{}'.format(epoch, batch_idx, loss.item()))

Train epoch:0, batch index:0, loss:2.303179979324341
Train epoch:0, batch index:1, loss:2.301939010620117
Train epoch:0, batch index:2, loss:2.30338716506958
Train epoch:0, batch index:3, loss:2.303199291229248
Train epoch:0, batch index:4, loss:2.3033149242401123
Train epoch:0, batch index:5, loss:2.3029682636260986
Train epoch:0, batch index:6, loss:2.302260160446167
Train epoch:0, batch index:7, loss:2.3016960620880127
Train epoch:0, batch index:8, loss:2.3026671409606934
Train epoch:0, batch index:9, loss:2.3028409481048584
Train epoch:0, batch index:10, loss:2.3027515411376953
Train epoch:0, batch index:11, loss:2.302941083908081
Train epoch:0, batch index:12, loss:2.3032407760620117
Train epoch:0, batch index:13, loss:2.302518367767334
Train epoch:0, batch index:14, loss:2.3017404079437256
Train epoch:0, batch index:15, loss:2.3033242225646973
Train epoch:0, batch index:16, loss:2.3031623363494873
Train epoch:0, batch index:17, loss:2.3026745319366455
Train epoch:0, batch index:1

In [17]:
def compute_metrics(dataloader):
    net.eval()
    loss = 0
    correct = 0

    confusion_matrix = np.zeros((10, 10))

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = net(data)
            loss += F.nll_loss(output, target, reduction="sum").item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred).sum().item())
            confusion_matrix[pred.cpu()[:, 0], target.cpu()] += 1

        loss = loss/len(test_loader.dataset)
        accuracy = 100.*correct/len(dataloader.dataset)

        return loss, accuracy, confusion_matrix